<a href="https://colab.research.google.com/github/siiddd/Risk-Analysis-Chatbot/blob/main/Final_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import pandas as pd
import numpy as np
import os
import re
import nltk
import pickle
import gzip

In [2]:
nltk.download('popular')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

True

In [3]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [4]:
dataset = pd.read_csv('https://raw.githubusercontent.com/siiddd/Risk-Analysis-Chatbot/main/dataset_augmented.csv')
dataset = dataset.iloc[:, [-2, -1]]
dataset.head(3)

,Description,Accident Level
0,at level access scissor performed the sustaini...,0
1,in general store while two store attendants re...,0
2,being p m approximately in the nv in the tecno...,3


In [5]:
description = dataset.iloc[:, -2]
description.head(3)

0    at level access scissor performed the sustaini...
1    in general store while two store attendants re...
2    being p m approximately in the nv in the tecno...
Name: Description, dtype: object

In [6]:
words = []

for i in range(len(description)):
  dataset_word = nltk.word_tokenize(description[i])
  words = [*words, *dataset_word]
  del dataset_word

In [7]:
words = list(set(words))
len(words)

4657

In [8]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip
!rm glove.6B.zip glove.6B.300d.txt glove.6B.50d.txt glove.6B.100d.txt 

--2022-03-26 03:54:53--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-03-26 03:54:53--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-03-26 03:54:53--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [56]:
os.getcwd()

'/content'

In [54]:
embeddings = {}

f = open('glove.6B.200d.txt', 'r', encoding = 'utf-8')
for line in f:
  values = line.split()
  words = values[0]
  coefs = np.asarray(values[1:], dtype = 'float32')
  embeddings[words] = coefs
f.close()

In [55]:
data_list = list()
for i in range(len(description)):
    sentence = np.zeros(200)
    count = 0
    dataset_word = nltk.word_tokenize(description[i])
    for x in dataset_word:
        try:
            sentence += embeddings[x]
            count += 1
        except KeyError:
            continue
    data_list.append(sentence / count)

In [60]:
len(data_list[0])

200

In [108]:
def brazil(problem):
  embeddings = {}

  f = open('glove.6B.200d.txt', 'r', encoding = 'utf-8')

  for line in f:
    values = line.split()
    words = values[0]
    coefs = np.asarray(values[1:], dtype = 'float32')
    embeddings[words] = coefs
  f.close()

  data_list = list()
  sentence = np.zeros(200)
  count = 0
  dataset_word = nltk.word_tokenize(description[i])
  for x in dataset_word:
       try:
           sentence += embeddings[x]
           count += 1
       except KeyError:
           continue
  data_list.append(sentence / count)
  return data_list[0]

In [142]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

model_RF =RandomForestClassifier()

model_RF.fit(data_list, dataset['Accident Level'])

pred = model_RF.predict(data_list)

print('Random Forest \nTraining accuracy', accuracy_score(dataset['Accident Level'], pred))

Random Forest 
Training accuracy 0.9983844911147012


In [120]:
import xgboost as xgb

model_XGB = xgb.XGBClassifier()

model_XGB.fit(pd.DataFrame(data_list), dataset['Accident Level'])

pred = model_XGB.predict(pd.DataFrame(data_list))

print('XGB Classifier: \nTraining accuracy', accuracy_score(dataset['Accident Level'], pred))

XGB Classifier: 
Training accuracy 0.9959612277867528


In [14]:
from sklearn.svm import SVC

model_SVM = SVC(kernel = 'linear')

model_SVM.fit(data_list, dataset['Accident Level'])

pred = model_SVM.predict(data_list)

print('Support Vector Machine \nTraining accuracy', accuracy_score(dataset['Accident Level'], pred))

Support Vector Machine 
Training accuracy 0.851373182552504


In [143]:
import pickle
model = open("model_RF.pkl","wb")
pickle.dump(model_RF, model)
model.close()

In [62]:
x = brazil('The sky is blue')

In [32]:
!wget https://github.com/siiddd/Risk-Analysis-Chatbot/blob/main/model_XGB.pkl

--2022-03-26 04:23:14--  https://github.com/siiddd/Risk-Analysis-Chatbot/blob/main/model_XGB.pkl
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘model_XGB.pkl’

model_XGB.pkl           [ <=>                ] 120.44K  --.-KB/s    in 0.08s   

2022-03-26 04:23:15 (1.57 MB/s) - ‘model_XGB.pkl’ saved [123330]



In [144]:
pickledmodel = open('model_RF.pkl',"rb")

In [145]:
nlp_classifier = pickle.load(pickledmodel)

In [146]:
nlp_classifier.predict(pd.DataFrame(x).T)

array([2])

In [102]:
!pip install streamlit

In [163]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip  
!unzip -qq ngrok-stable-linux-amd64.zip

--2022-03-26 09:28:47--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.237.133.81, 54.161.241.46, 52.202.168.65, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.237.133.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.1’

ngrok-stable-linux- 100%[===================>]  13.19M  36.1MB/s    in 0.4s    

2022-03-26 09:28:47 (36.1 MB/s) - ‘ngrok-stable-linux-amd64.zip.1’ saved [13832437/13832437]

replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y


In [165]:
%%writefile risk.py

import numpy as np
import pickle
import pandas as pd
import streamlit as st
from PIL import Image
import pandas as pd
import nltk

pickledmodel = open('model_RF.pkl',"rb")
nlp_classifier = pickle.load(pickledmodel)

#model_XGB = xgboost.Booster()
#model_XGB.load_model('model_XGB.JSON')


def brazil(problem):
  embeddings = {}

  f = open('glove.6B.200d.txt', 'r', encoding = 'utf-8')

  for line in f:
    values = line.split()
    words = values[0]
    coefs = np.asarray(values[1:], dtype = 'float32')
    embeddings[words] = coefs
  f.close()

  data_list = list()
  sentence = np.zeros(200)
  count = 0
  dataset_word = nltk.word_tokenize(problem)
  for x in dataset_word:
       try:
           sentence += embeddings[x]
           count += 1
       except KeyError:
           continue
  data_list.append(sentence / count)
  return data_list[0]


def final(x):
    
    prediction = model_RF.predict(pd.DataFrame(x).T)
    print(prediction)
    return prediction



def main():
    st.title("Risk Analysis Chatbot")
    html_temp = """
    <div style="background-color:tomato;padding:10px">
    <h2 style="color:white;text-align:center;"> Risk Analysis Chatbot </h2>
    </div>
    """
    st.markdown(html_temp,unsafe_allow_html=True)
    problem = st.text_input("Problem","Type Here")

    result=""

    if st.button("Predict"):
        x = brazil(problem)
        result = final(x)
    st.success('The output is {}'.format(result))

if __name__=='__main__':
    main()

Overwriting risk.py


In [172]:
!streamlit run risk.py

2022-03-26 09:37:58.135 INFO    numexpr.utils: NumExpr defaulting to 2 threads.

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://34.133.15.155:8501

  Stopping...
  Stopping...


In [174]:
#you may have to run this cell again if it throws an erro
get_ipython().system_raw('./ngrok http 8501 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"
    
# this gives you the link on which your web app will be deployed

http://f3ee-34-133-15-155.ngrok.io
